<a href="https://colab.research.google.com/github/Bigote93/Chatbot_Alura_Sprint1/blob/main/Chatbot_Inteligente_Alura_Sprint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint 01 - Chatbot - Alura Latam

In [20]:
#Recursos y librerias
import pandas as pd
import re, os, random, pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

!pip install scipy
import spacy

!pip install jellyfish
import jellyfish

!pip install transformers

from transformers import BertForSequenceClassification
from transformers import BertTokenizer

import torch



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## (1) Lista de carga de verbos

In [3]:
#Instanciamos la lista de verbos
verbos_irregulares = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    verbos_irregulares = pickle.load(fichero_listaverbos)

In [4]:
#Instanciamos la lista de verbos
lista_verbos = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    lista_verbos = pickle.load(fichero_listaverbos)

## (2) Tratamiento de datos

### Raiz de los verbos

Función raiz que reciba una palabra y la compare con todas las palabras de la lista_verbos utilizando jaro_winkler , y que devuelva la palabra de lista_verbos con mayor similaridad a la palabra ingresada.

> Observación: Si la palabra encontrada, con mayor similaridad, no supera el radio de 0.93 entonces deberá regresar la palabra original.

In [5]:
def raiz_verbo(verbo_analizar):

  #Lista de verbos cercanos
  verbos_cercanos = []

  #Recorremos la lista de verbos
  for verbo in lista_verbos:
    #Condicionador de verbos y adjuntar a la lista
    if jellyfish.jaro_winkler_similarity(verbo_analizar,verbo) >= 0.93:
      #Verificamos si esta el verbo en la lista
      if verbo not in verbos_cercanos:
        verbos_cercanos.append(verbo)
    else:
      #Verificamos si esta el verbo en la lista
      if verbo_analizar not in verbos_cercanos:
        verbos_cercanos.append(verbo_analizar)

  return verbos_cercanos

#Aplicacion
raiz_verbo('Comir')

['Comir']

### Tratamiento de texto

Función tratamiento_texto que reciba una frase de texto y devuelva la misma frase pero sin acentuaciones, todo en minúscula y sin espacios en blanco adicionales.

Ejemplo:

```
# Ejemplo visualizacion

tratamiento_texto('¡Buen día!, ¿Cómo está todo hoy?')

>> '¡buen dia!, ¿como esta todo hoy?'

```



In [6]:
def tratamiento_texto(texto):

  # Convertimos el texto a minúsculas.
  texto_minusculas = texto.lower()

  # Eliminamos los caracteres con tildes.
  texto_tratado = texto_minusculas.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

  return texto_tratado

tratamiento_texto('¡Buen día!, ¿Cómo está todo hoy?')

'¡buen dia!, ¿como esta todo hoy?'

### Terminacion de palabras

Función reemplazar_terminacion que reciba una palabra e identifique si la misma termina en alguna de las siguientes palabras: “es”, “me”, “as”, “te”, “ste”, si coincide, entonces que substituya esa terminación por la letra “r”.

In [7]:
def reemplazar_terminacion(palabra):

  terminaciones = ['ste','es','me','as','te',]
  #Verificar si tiene terminacion
  for terminacion in terminaciones:
    if terminacion in palabra[-len(terminacion):]:

      #Cambiar la terminacion de la palabra
      verbo_infinitivo = palabra[:-len(terminacion)]+'r'

      return verbo_infinitivo

reemplazar_terminacion('Comiste')

'Comir'

### Normalizacion de textos

Crear una funcion que identifique la categoria de la palabra.

>Observacion: Antes de realizarlo, debemos aplicar la funcion de puntuacion y tildes para limpiar la oracion

In [8]:
import spacy

def normalizar_texto(frase):

  #Instanciar modelo nlp
  nlp = spacy.load('en_core_web_sm')

  #Aplicamos la instancia a la frase
  doc = nlp(tratamiento_texto(frase))

  #Barrimos token de la frase para determinar categoria de token
  for token in doc:
    print(token.text, '-', token.pos_ , '-', token.lemma_)


normalizar_texto('Hola, mi nombre es Diego Nalli')

hola - PROPN - hola
, - PUNCT - ,
mi - PROPN - mi
nombre - PROPN - nombre
es - PROPN - es
diego - PROPN - diego
nalli - PROPN - nalli


## (3) Cargar base de documentos

### Cargar preguntas

In [9]:
# Generamos una ruta relativa para busqueda de elementos
direccion_relativa = '/content/drive/MyDrive/Chatbot'
dir_relativa_entrenamiento = direccion_relativa + '/dialogos'

# Barrido de documentos internos el folder
titulo_doc_list = [documento for documento in os.listdir(dir_relativa_entrenamiento) if documento.endswith('.txt')]

# Creamos listas para almacenar contenidos
lista_dialogos, lista_dialogos_respuesta, lista_dialogos_tipo = [], [], []

#Recorremos los documentos para almacenar las oraciones
for name_doc in range(len(titulo_doc_list)):

  # Apertura de documentos con los titulos almacenados
  doc = open(
      dir_relativa_entrenamiento + '/' + titulo_doc_list[name_doc],
      'r',
      encoding = 'utf-8',
      errors = 'ignore'
      )

  # Separar en lineas por cada texto
  lineas = doc.read().split('\n')

  # Carga de lineas del documento abierto
  for index, linea in enumerate(lineas):



    if index % 2 != 0: # Par: Respuesta necesaria
      lista_dialogos_respuesta.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              )
          )

      # Almacenar el tipo de linea
      lista_dialogos_tipo.append(titulo_doc_list[name_doc].replace('.txt', ''))

    else: # Impar: Pregunta a bot
      lista_dialogos.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              ))
print(f'El tamaño de dialogos es: {len(lista_dialogos)}')
print(f'El tamaño de respuestas es: {len(lista_dialogos_respuesta)}')
print(f'El tamaño de tipo es: {len(lista_dialogos_tipo)}')

El tamaño de dialogos es: 1152
El tamaño de respuestas es: 1152
El tamaño de tipo es: 1152


In [10]:
# Creamos dataframe "df_dialogo"
datos_dialogo = {
    'dialogo': lista_dialogos,
    'respuesta': lista_dialogos_respuesta,
    'tipo': lista_dialogos_tipo,
    'interseccion': 0,
    'jaro_winkler': 0,
    'probabilidad': 0
}

#Instanciamos el dataframe
df_dialogo = pd.DataFrame(datos_dialogo)

#Eliminamos los registros que se han repetido
df_dialogo = df_dialogo.drop_duplicates(keep = 'first')

#Eliminamos el index del dataframe
df_dialogo.reset_index(drop=True, inplace=True)

df_dialogo.sample(5)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
854,pero por que pusiste un dibujo en tu foto de p...,mi eleccion de poner un dibujo en mi foto de p...,Otros,0,0,0
945,usando la definicion que te di,no puedo hacerlo solo puedo ser re-entrenado p...,Otros,0,0,0
842,y puedes redactar un texto por mi,no estoy programado para ayudarte con esto sol...,Otros,0,0,0
958,me interesa curso de programacion,debes revisar el canal de elprofealejo en youtube,Otros,0,0,0
1111,gracias por tu asistencia,siempre estoy aqui para ayudarte en todo lo qu...,Agradecimiento,0,0,0


## (4) Buscar respuesta del Chatbot

### Comparacion de Textos

In [ ]:
# Funcion para verificar si el usuario inicio un dialogo
def dialogo(pregunta_usuario, df_registros):

  # Al igual que la normalizacion de preguntas, eliminamos caracteres como normalizados
  pregunta_user_tratada = re.sub(
      r"[^\w\s]",
      '',
      tratamiento_texto(pregunta_usuario)
      )

  # Crear un vectorizador y entrenarlo con tus datos
  vectorizador = TfidfVectorizer()
  tfidf_matrix = vectorizador.fit_transform(df_dialogo['dialogo'])

  # Funcion interseccion
  def interseccion(texto_user, texto_comparar):

    #Transformamos a Set un string
    user_text = set(texto_user)
    df_text = set(texto_comparar)

    # Calculo de interseccion
    per_words_commun = len(list(user_text & df_text)) / len(list(user_text))


    return per_words_commun

  # Funcion similarity
  def similarity(texto_user, texto_comparar, tfidf_matrix):

    # Transformamos y obtenemos la union entre los registros
    text_user_vector = vectorizador.transform([texto_user])

     # Calcular la similitud coseno entre el texto del usuario y el texto de comparación
    similarity_score = cosine_similarity(text_user_vector, tfidf_matrix)

    return similarity_score[0][0]


  # Recorremos cada fila para buscar cercania de pregunta con dialogo
  for idx, row in df_registros.iterrows():
    df_registros.at[idx, 'interseccion'] = interseccion(pregunta_user_tratada, row)
    df_registros.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(pregunta_user_tratada,row)
    df_registros.at[idx, 'similarity'] = ''


In [47]:
# Crear un vectorizador y entrenarlo con tus datos
vectorizador = TfidfVectorizer()
tfidf_matrix = vectorizador.fit_transform(df_dialogo.dialogo)

# Cadenas de texto
texto1 = 'hola como estas'
texto2 = 'muy bien y tu'

# Funcion similarity
def similarity(texto_user, texto_comparar, tfidf_matrix):

  # Transformamos y obtenemos la union entre los registros
  text_user_vector = vectorizador.transform([texto_user, texto_comparar])

  # Calcular la similitud coseno entre el texto del usuario y el texto de comparación
  similarity_score = cosine_similarity(text_user_vector,tfidf_matrix )

  return similarity_score[0][0]

similarity(texto1, texto2,tfidf_matrix)


0.0

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cadenas de texto
texto1 = 'diego nalli garcia'
texto2 = 'alfredo nalli urtubia'

# Funcion similarity
def similarity(texto_user, texto_comparar):

  # Iniciar vectorizador
  vectorizador =  TfidfVectorizer()

  # Entrenamos vectorizador
  matriz_tfid = vectorizador.fit_transform(df_dialogo.dialogo)

  # Transformamos y obtenemos la union entre los registros
  text_user_registro = vectorizador.transform([texto_user])

  return text_user_registro

print(similarity(texto1, texto2))